In [47]:
from pathlib import Path
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import soundfile as sf
import pandas as pd

import os 
import itertools
import re
import math
import csv
import random
from sklearn import tree
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn import linear_model
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.datasets import make_classification
from sklearn.ensemble import GradientBoostingClassifier
from scipy.fftpack import fftn, ifftn, fft, ifft
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from scipy import signal
import pickle
from sklearn.externals import joblib
from sklearn.datasets import load_digits
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit
import warnings

warnings.filterwarnings('ignore') 
df = pd.read_csv("Features_all.csv")
df = df[(df.modeID != 'audible' )]
df

,channel_1_mav,channel_1_rms,channel_1_iav,channel_1_var,channel_1_ssi,channel_1_var.1,channel_1_wl,channel_1_iemg,channel_1_aac,channel_1_zc,...,channel_6_aac,channel_6_zc,channel_6_ssc,channel_6_wamp,channel_6_medf,channel_6_meanf,userID,sessionID,uttID,modeID
50,0.302146,0.428303,855.071838,0.178186,519.144666,0.178186,1256.635712,855.071838,0.444041,872,...,0.444218,874,1624,789,112.765957,117.610824,1,1001,10010201,silent
51,0.300652,0.427435,1234.777893,0.177518,750.350798,0.177518,1823.810242,1234.777893,0.444074,1269,...,0.443859,1297,2357,1125,124.694377,124.815305,1,1001,10010202,silent
52,0.301546,0.427589,724.613983,0.177686,439.345396,0.177686,1066.772095,724.613983,0.443933,714,...,0.442849,720,1374,662,94.142259,110.282368,1,1001,10010203,silent
53,0.301181,0.427158,1138.764557,0.177491,689.897471,0.177491,1681.027466,1138.764557,0.444599,1174,...,0.444029,1144,2160,1056,84.350133,106.473499,1,1001,10010204,silent
54,0.302001,0.428487,1338.771912,0.178612,813.903190,0.178612,1977.487030,1338.771912,0.446083,1356,...,0.445143,1366,2538,1230,120.814480,122.660870,1,1001,10010205,silent
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6865,0.207974,0.385068,566.728729,0.137972,404.055786,0.137972,1027.747131,566.728729,0.377155,1677,...,0.375292,1702,2214,389,66.420664,98.458670,8,8019,80190246,whisper
6866,0.206211,0.384257,458.200958,0.137259,328.085320,0.137259,834.120422,458.200958,0.375392,1372,...,0.374308,1374,1823,317,35.294118,89.446197,8,8019,80190247,whisper
6867,0.205649,0.384072,540.651550,0.137091,387.806500,0.137091,982.714752,540.651550,0.373798,1630,...,0.373218,1628,2147,375,44.827586,92.387261,8,8019,80190248,whisper
6868,0.205896,0.384267,411.173523,0.137218,294.878605,0.137218,746.980469,411.173523,0.374051,1245,...,0.374330,1232,1644,285,21.212121,83.388139,8,8019,80190249,whisper


In [52]:
def split_data_per_session(session_id):
    temp_df = df[(df.sessionID == session_id)]
    data = temp_df.copy()
    data = shuffle(data)
    y = data['modeID']
    data.drop(labels=['userID','uttID', 'sessionID','modeID'], axis=1, inplace=True)
    return train_test_split(data, y, test_size=0.2)

def split_data_session(train_user_ids,test_user_ids):
    temp_df = df
    data = temp_df.copy()
    train_df = data.loc[data['sessionID'].isin(train_user_ids)]
    test_df = data.loc[data['sessionID'].isin(test_user_ids)]
    X_train = train_df.copy()
    X_test = test_df.copy()
    y_train = X_train['modeID']
    y_test = X_test['modeID']
    X_train.drop(labels=['userID','uttID', 'sessionID','modeID'], axis=1, inplace=True)
    X_test.drop(labels=['userID','uttID', 'sessionID','modeID'], axis=1, inplace=True)
    return X_train, X_test, y_train, y_test 

def split_data_utterance(train_user_ids,test_user_ids):
    temp_df = df
    data = temp_df.copy()
    train_df = data.loc[data['userID'].isin(train_user_ids)]
    test_df = data.loc[data['userID'].isin(test_user_ids)]
    X_train = train_df.copy()
    X_test = test_df.copy()
    y_train = X_train['modeID']
    y_test = X_test['modeID']
    X_train.drop(labels=['userID','uttID', 'sessionID','modeID'], axis=1, inplace=True)
    X_test.drop(labels=['userID','uttID', 'sessionID','modeID'], axis=1, inplace=True)
    return X_train, X_test, y_train, y_test 

def split_data(train_user_ids,test_user_ids):
    temp_df = df
    data = temp_df.copy()
    train_df = data.loc[data['userID'].isin(train_user_ids)]
    test_df = data.loc[data['userID'].isin(test_user_ids)]
    X_train = train_df.copy()
    X_test = test_df.copy()
    y_train = X_train['modeID']
    y_test = X_test['modeID']
    X_train.drop(labels=['userID','uttID', 'sessionID','modeID'], axis=1, inplace=True)
    X_test.drop(labels=['userID','uttID', 'sessionID','modeID'], axis=1, inplace=True)
    return X_train, X_test, y_train, y_test 

In [62]:
lda_all_sessions()

Accuracy for all users is 0.68
Accuracy: 0.54 (+/- 0.32)


In [63]:
lda_all()

Accuracy for all users is 0.5257142857142857
Accuracy: 0.66 (+/- 0.09)
